# Transfer Learning

# 1. Ready

## 1.1 기본 패키지 임포트

In [1]:
import torch
from torch.utils.data import DataLoader


## 1.2 CUDA Check

In [2]:
if torch.cuda.device_count():
  print('된다!')
  PU = 'cuda'
else:
  PU = 'cpu'

된다!


## 1.3 Download Pretrained Ko-GPT2

### KoGPT2 Clone

In [3]:
!git clone https://github.com/SKT-AI/KoGPT2.git

fatal: destination path 'KoGPT2' already exists and is not an empty directory.


### Tokenizer, Model download 

In [4]:
MODEL_NAME = "skt/kogpt2-base-v2"
save_path = './modelCheckpoint/'

In [5]:
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel

tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_NAME)
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)
TOKENS_DICT = {
    'bos_token':'<s>',
    'eos_token':'</s>',
    'unk_token':'<unk>',
    'pad_token':'<pad>',
    'mask_token':'<mask>'
}

# 특수 토큰이 토크나이저에 추가되고 모델은 수정된 토크나이저에 맞게 임베딩의 크기를 조정
tokenizer.add_special_tokens(TOKENS_DICT)
model.resize_token_embeddings(len(tokenizer))

print(tokenizer.special_tokens_map)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


{'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'}


# 2. 데이터 세팅

In [6]:
from util.data_setter import Data_Set

In [7]:
file_path = './data/정제/merged_data.txt'
data = Data_Set(file_path, tokenizer)
dataset = DataLoader(data, batch_size=2, shuffle=True, pin_memory=True)

# 3. 모델 학습

## 3.1 모델 세팅

### * 학습모델이 있는 경우만 loading=True로 하고 실행

In [8]:
from util.model import model_loading as load

In [24]:
checkpointPath = "./modelCheckpoint/checkpointmodel3.tar"
loading = False

In [ ]:
model,checkpoint = load(checkpointPath, PU = PU, status = loading)

## 3.2 하이퍼파라미터 설정

### Note
- Optimizer = Adam 사용
- Learning_rate = Epoch에 따라 감소시키자

In [22]:
learning_rate = 0.00005
epochs = 300
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
current = [0 if loading==False else checkpoint['epoch']][0]

In [18]:
save_path

'./modelCheckpoint/'

In [23]:
for epoch in range(current+1, epochs+1):
    cnt = 0

    for data in dataset:
        optimizer.zero_grad()

        data = torch.stack(data)
        data = data.transpose(1,0)
        data = data.to(PU)

        try: 
            output = model(data,labels=data)
        except:
            print("Size Error => Passed")
            continue
        loss, logits = output[:2]
        loss.backward()
        optimizer.step()

        if cnt % 20 == 0:
            print("[+] epoch : {}, cnt : {}, loss : {} [+]".format(epoch, cnt+1, str(loss)[7:12]))

        if epoch % 10 == 0 and cnt == 1:
            print('Saving Model.....')
            torch.save({
              'epoch': epoch,
              'cnt': cnt,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': loss,
            }, save_path+'checkpointmodel4.tar')
      
        cnt += 1

NameError: name 'optimizer' is not defined